In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('C:\\Users\\qml\\Desktop\\学习\\reinforcement learning for Fintech\\Ubiquant Market Prediction\\train.csv')

In [3]:
featurenamelist=list(np.load(r'C:\Users\qml\Desktop\学习\reinforcement learning for Fintech\Ubiquant Market Prediction\dropout+normrlbatch+250feature\featurename.npy'))

In [4]:
feature = df[featurenamelist]
y = df["target"]
id=df["investment_id"]

In [5]:
import torch
from torch.utils.data import DataLoader,Dataset

In [6]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
feature=torch.from_numpy(feature.values)
id=torch.from_numpy(id.values)
y=torch.from_numpy(y.values)
class FinData(Dataset):
    def __init__(self,feature,id,y):
        self.feature = feature.squeeze().to("cuda")
        self.y = y.squeeze().to("cuda")
        self.id = id.squeeze().to(torch.int64).to("cuda")
        self.len = len(y)

    def __getitem__(self,idx):
        return self.id[idx],self.feature[idx],self.y[idx]
  
    def __len__(self):
        return self.len
Dataset_train=FinData(feature=feature,id=id,y=y)
DataLoader1=DataLoader(Dataset_train,batch_size=128,shuffle=True)
DataLoader2=DataLoader(Dataset_train,batch_size=1024,shuffle=True)
DataLoader3=DataLoader(Dataset_train,batch_size=512,shuffle=True)
DataLoader4=DataLoader(Dataset_train,batch_size=256,shuffle=True)


In [7]:
from torch import nn
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.embedding=nn.Embedding(4000,32)
        self.linear1=nn.Linear(32,64)
        self.linear2=nn.Linear(64,64)
        self.linear3=nn.Linear(250,256)
        self.linear4=nn.Linear(256,256)
        self.linear5=nn.Linear(320,512)
        self.linear6=nn.Linear(512,128)
        self.linear7=nn.Linear(128,32)
        self.linear8=nn.Linear(32,1)
        self.linear9=nn.Linear(32,2)
        self.dropout=nn.Dropout(p=0.1)
      

    def forward(self, feature, id):
        id=self.embedding(id).squeeze()
        id=self.linear1(id).squeeze()
        id=self.dropout(id)
        id=id*torch.sigmoid(id)
        id=self.linear2(id).squeeze()
        id=id*torch.sigmoid(id)
        id=self.linear2(id).squeeze()
        id=id*torch.sigmoid(id)


        feature=self.linear3(feature).squeeze()
        feature=self.dropout(feature)
        feature=feature*torch.sigmoid(feature)
        feature=self.linear4(feature).squeeze()
        feature=feature*torch.sigmoid(feature)
        feature=self.linear4(feature).squeeze()
        feature=feature*torch.sigmoid(feature)

        x_all=torch.cat((id,feature),1)
        x=self.linear5(x_all).squeeze()
        x=x*torch.sigmoid(x)
        x=self.linear6(x).squeeze()
        x=x*torch.sigmoid(x)

        x=self.linear7(x).squeeze()
        x=x*torch.sigmoid(x)

        x=self.linear8(x).squeeze()
        x=20*torch.sigmoid(x)


        x2=self.linear5(x_all).squeeze()
        x2=x2*torch.sigmoid(x2)

        x2=self.linear6(x2).squeeze()
        x2=x2*torch.sigmoid(x2)

        x2=self.linear7(x2).squeeze()
        x2=x2*torch.sigmoid(x2)

        x2=self.linear9(x2).squeeze()
        x2=torch.sigmoid(x2)














        return x.squeeze(),x2.squeeze()

In [8]:
model1_1 = NeuralNetwork().to("cuda")
model1_1=model1_1.double()
model1_2 = NeuralNetwork().to("cuda")
model1_2=model1_2.double()

model2_1 = NeuralNetwork().to("cuda")
model2_1=model2_1.double()
model2_2 = NeuralNetwork().to("cuda")
model2_2=model2_2.double()


model3_1 = NeuralNetwork().to("cuda")
model3_1=model3_1.double()
model3_2 = NeuralNetwork().to("cuda")
model3_2=model3_2.double()

model4_1 = NeuralNetwork().to("cuda")
model4_1=model4_1.double()
model4_2 = NeuralNetwork().to("cuda")
model4_2=model4_2.double()

In [9]:
model1_1=torch.load('model1_1.pth')
model1_2=torch.load('model1_2.pth')
model2_1=torch.load('model2_1.pth')
model2_2=torch.load('model2_2.pth')
model3_1=torch.load('model3_1.pth')
model3_2=torch.load('model3_2.pth')
model4_1=torch.load('model4_1.pth')
model4_2=torch.load('model4_2.pth')

In [10]:
learning_rate = 1e-3
optimizer1_1 = torch.optim.Adam(model1_1.parameters(), lr=learning_rate,weight_decay=1e-5)
optimizer1_2 = torch.optim.Adam(model1_2.parameters(), lr=learning_rate,weight_decay=1e-5)
optimizer2_1 = torch.optim.Adam(model2_1.parameters(), lr=learning_rate,weight_decay=1e-5)
optimizer2_2 = torch.optim.Adam(model2_2.parameters(), lr=learning_rate,weight_decay=1e-5)
optimizer3_1 = torch.optim.Adam(model3_1.parameters(), lr=learning_rate,weight_decay=1e-5)
optimizer3_2 = torch.optim.Adam(model3_2.parameters(), lr=learning_rate,weight_decay=1e-5)
optimizer4_1 = torch.optim.Adam(model4_1.parameters(), lr=learning_rate,weight_decay=1e-5)
optimizer4_2 = torch.optim.Adam(model4_2.parameters(), lr=learning_rate,weight_decay=1e-5)
loss_fn1=torch.nn.MSELoss()
loss_fn2=torch.nn.CrossEntropyLoss()

In [11]:
from tqdm import tqdm

In [12]:
def train_1(dataloader, model, optimizer):
    for  id,feature,y in tqdm(dataloader):
        # Compute prediction and loss
        pred,label = model(feature,id)
        loss1 = loss_fn1(pred,torch.abs(y))
        loss2 = loss_fn2(label,(0.5*(y/torch.abs(y)+torch.tensor([1]).to(device))).long().to("cuda"))
        # Backpropagation
        loss = loss1+0.05*loss2
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
def train_2(dataloader, model, optimizer):
    for id,feature, y in tqdm(dataloader):
        pred,label = model(feature,id)
        loss1 = -torch.corrcoef(torch.stack((pred,torch.abs(y))))[0,1]
        loss2 = loss_fn2(label,(0.5*(y/torch.abs(y)+torch.tensor([1]).to(device))).long().to("cuda"))
        # Backpropagation
        loss = loss1+0.05*loss2
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        

        # Compute prediction and loss
        # pred = model(feature,id)
        # loss = -torch.corrcoef(torch.stack((pred,y)))[0,1]
        # # Backpropagation
        # optimizer.zero_grad()
        # loss.backward()
        # optimizer.step()

In [13]:
def test(model, dataloader):
    score=0
    number=0
    for id, feature,y in tqdm(dataloader):
        number+=1
        with torch.no_grad():
            pred,label=model(feature,id)
            cats=[]
            for cat in label:
                if cat[0]>cat[1]:
                    cats.append(-1)
                else:
                    cats.append(1)
            cats=torch.tensor(cats).to(device)
            pred=pred*cats
            score+=torch.corrcoef(torch.stack((pred,y)))[0,1].item()
    score=score/number
    print(score)            


In [14]:
number_epoch=20
for i in range(number_epoch):
    print("epoch{}/{}".format(i+1,number_epoch))
    train_1(DataLoader1,model1_1,optimizer1_1)
    train_2(DataLoader1,model1_2,optimizer1_2)


    train_1(DataLoader2,model2_1,optimizer2_1)
    train_2(DataLoader2,model2_2,optimizer2_2)


    train_1(DataLoader3,model3_1,optimizer3_1)
    train_2(DataLoader3,model3_2,optimizer3_2)


    train_1(DataLoader4,model4_1,optimizer4_1)
    train_2(DataLoader4,model4_2,optimizer4_2)

    if (i+1)%5==0:
        test(model1_1,DataLoader2)
        test(model1_2,DataLoader2)
        test(model2_1,DataLoader2)
        test(model2_2,DataLoader2)
        test(model3_1,DataLoader2)
        test(model3_2,DataLoader2)
        test(model4_1,DataLoader2)
        test(model4_2,DataLoader2)

epoch1/20


100%|██████████| 12272/12272 [02:07<00:00, 96.36it/s] 


epoch2/20


100%|██████████| 12272/12272 [02:16<00:00, 89.81it/s]


epoch3/20


...

0.06363341676028413


100%|██████████| 3068/3068 [03:21<00:00, 15.22it/s]


0.05959566367771941


 67%|██████▋   | 2060/3068 [02:15<01:06, 15.18it/s]


KeyboardInterrupt: 

In [ ]:
torch.save(model1_1, 'model1_1.pth')
torch.save(model1_2, 'model1_2.pth')
torch.save(model2_1, 'model2_1.pth')
torch.save(model2_2, 'model2_2.pth')
torch.save(model3_1, 'model3_1.pth')
torch.save(model3_2, 'model3_2.pth')
torch.save(model4_1, 'model4_1.pth')
torch.save(model4_2, 'model4_2.pth')